In [2]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time 
import sys   
import math
import pandas as pd
import os
import urllib.request
import re
import random
from urllib.parse import urljoin

print("=" * 80)
print("아마존 폴란드 Best Seller 상품 정보 추출하기 ")
print("=" * 80)

# 검색 설정
query_txt = '아마존'

# 폴란드 아마존 베스트셀러 URL (필요시 변경)
query_url = 'https://www.amazon.pl/gp/bestsellers/beauty/ref=zg_bs_unv_beauty_1_20805439031_1'
base_url = "https://www.amazon.pl"

# 수집 건수 및 페이지 수 계산
cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 50)

# 저장 폴더 설정
f_dir = input('2.파일을 저장할 폴더명만 쓰세요(기본값:c:\\py_temp2\\):')
if f_dir == '':
    f_dir = "c:\\py_temp2\\"

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (
    now.tm_year, now.tm_mon, now.tm_mday,
    now.tm_hour, now.tm_min, now.tm_sec
)

# 작업 폴더 및 이미지 폴더 생성
work_dir = os.path.join(f_dir, s + '-' + query_txt)
img_dir = os.path.join(work_dir, 'images')

os.makedirs(img_dir, exist_ok=True)
os.chdir(work_dir)

ff_name = os.path.join(work_dir, f'{s}-{query_txt}.txt')
fc_name = os.path.join(work_dir, f'{s}-{query_txt}.csv')
fx_name = os.path.join(work_dir, f'{s}-{query_txt}.xlsx')

# 크롬 옵션 설정
options = Options()
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/120.0.0.0 Safari/537.36"
)
options.add_argument("--lang=ko-KR")

# 자동 번역 옵션 – 필요없으면 주석 처리
options.add_experimental_option('prefs', {
    "translate_whitelists": {"pl": "en"},
    "translate": {"enabled": True}
})

# 크롬 드라이버 실행
time.sleep(1)
driver = webdriver.Chrome(options=options)

driver.get(query_url)
time.sleep(random.uniform(2, 4))
driver.maximize_window()
time.sleep(random.uniform(2, 4))


# 스크롤 함수
def scroll_down(drv):
    drv.execute_script("window.scrollBy(0, 1000);")
    time.sleep(random.uniform(2.5, 4.5))


# 수집 내용을 저장하기 위해 빈 리스트를 지정
ranking2 = []     # 판매순위
url2 = []         # 상품 URL
title2 = []       # 상품명
s_price2 = []     # 현재 판매가격
star_point2 = []  # 별점
review_cnt2 = []  # 리뷰 개수
content2 = []     # 상세내용 1
content_2 = []    # 상세내용 2

# 이미지 파일 번호 (엑셀 삽입용)
img_file_no = 1

# 전체 수집 개수
total_cnt = 0

f = open(ff_name, 'a', encoding='UTF-8')

for x in range(1, page_cnt + 1):

    if total_cnt >= cnt:
        break

    for _ in range(5):
        scroll_down(driver)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 베스트셀러 목록 찾아오기
    ol_tag = soup.find('ol', class_='a-ordered-list a-vertical p13n-gridRow _cDEzb_grid-row_3Cywl')
    if not ol_tag:
        print("상품 리스트를 찾지 못했습니다. (레이아웃 변경 가능성)")
        break

    slist = ol_tag.find_all('li', class_="zg-no-numbers")
    time.sleep(random.uniform(3, 5))

    # 페이지 내 순번
    page_rank = 1

    for li in slist:

        if total_cnt >= cnt:
            break

        # 이미지 저장
        try:
            img_span = li.find('span', class_='a-list-item')
            photo = img_span.find('img')['src']
        except Exception:
            continue

        img_path = os.path.join(img_dir, f"{img_file_no}.jpg")
        try:
            urllib.request.urlretrieve(photo, img_path)
            time.sleep(random.uniform(1, 2))
        except Exception:
            continue

        print("-" * 70)
        f.write("-----------------------------------------------------\n")

        # URL
        try:
            href = li.find("a")["href"]
            url = urljoin(base_url, href)
        except Exception:
            continue

        print('1.url:', url.replace("\n", ""))
        f.write('1.url:' + url + "\n")
        url2.append(url)

        # 판매순위
        try:
            ranking = li.find('span', class_='zg-bdg-text').get_text().strip()
            ranking_clean = ranking.replace("#", "").strip()
        except Exception:
            ranking_clean = str(page_rank)

        print('2.판매순위:', ranking_clean)
        f.write('2.판매순위:' + ranking_clean + "\n")
        ranking2.append(ranking_clean)

        # 상품명
        try:
            title = li.find('div', class_='_cDEzb_p13n-sc-css-line-clamp-3_g3dy1').get_text().strip()
        except Exception:
            try:
                title = li.find('div', class_='_cDEzb_p13n-sc-css-line-clamp-4_2q2cc').get_text().strip()
            except Exception:
                title = ""

        print("3.제품이름:", title.replace("\n", ""))
        f.write('3.제품이름:' + title + "\n")
        title2.append(title.replace("\n", ""))

        # 판매 가격
        try:
            s_price = li.find('span', class_='_cDEzb_p13n-sc-price_3mJ9Z').get_text().strip()
        except Exception:
            s_price = "가격 없음"

        print("4.판매가격:", s_price.replace("\n", ""))
        f.write('4.판매가격:' + s_price + "\n")
        s_price2.append(s_price.replace("\n", ""))

        # 별점
        try:
            star_point = li.find('span', class_='a-icon-alt').get_text()
            m = re.search(r"(\d+[.,]\d+)", star_point)
            if m:
                star_point_1 = m.group(1).replace(",", ".")
            else:
                star_point_1 = ""
        except Exception:
            star_point_1 = ""

        print("5.별점:", star_point_1)
        f.write('5.별점:' + star_point_1 + "\n")
        star_point2.append(star_point_1)

        # 리뷰 개수
        try:
            review_span = li.find("div", class_="a-icon-row").find("span", class_="a-size-small")
            review_cnt = review_span.get_text().strip()
        except Exception:
            review_cnt = "0"

        print("6.리뷰개수:", review_cnt.replace("\n", ""))
        f.write("6.리뷰개수:" + review_cnt + "\n")
        review_cnt2.append(review_cnt.replace("\n", ""))

        # 상세 페이지 열어서 로그인/보안 페이지 감지 
        content = ""
        content_1 = ""

        try:
            # 새 탭으로 상세 페이지 열기
            driver.execute_script("window.open(arguments[0]);", url)
            time.sleep(random.uniform(2, 4))
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(random.uniform(3, 5))

            current_url = driver.current_url

            # 로그인/보안 페이지 감지
            if ("signin" in current_url) or ("ap/signin" in current_url) or ("ap/cvf" in current_url):
                print("   → 로그인/보안 페이지 감지: 상세내용은 빈 값으로 두고 건너뜀")
            else:
                html2 = driver.page_source
                soup2 = BeautifulSoup(html2, 'html.parser')
                time.sleep(random.uniform(2, 3))

                # 상세내용 1
                try:
                    ul_tag = soup2.find("ul", class_="a-unordered-list a-vertical a-spacing-mini")
                    if ul_tag:
                        content = ul_tag.get_text(separator=" ").strip()
                    else:
                        content = ""
                except Exception:
                    content = ""

                print("7.상세내용:", content.replace("\n", ""))
                f.write('7.상세내용:' + content + "\n")
                content2.append(content.replace("\n", ""))

                # 상세내용 2
                try:
                    div_tag = soup2.find("div", class_="a-section a-spacing-small a-spacing-top-small")
                    if div_tag:
                        content_1 = div_tag.get_text(separator=" ").strip()
                    else:
                        content_1 = ""
                except Exception:
                    content_1 = ""

                print("8.상세내용2:", content_1.replace("\n", ""))
                f.write('8.상세내용2:' + content_1 + "\n")
                content_2.append(content_1.replace("\n", ""))

        except Exception as e:
            print("   → 상세 페이지 처리 중 예외 발생:", e)
            content2.append("")
            content_2.append("")

        # 상세 탭 닫고 원래 창으로 돌아오기
        try:
            driver.close()
            time.sleep(random.uniform(1, 2))
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(random.uniform(1, 2))
        except Exception:
            pass

        img_file_no += 1
        total_cnt += 1
        page_rank += 1


        # 50개마다 30초 휴식
        if total_cnt > 0 and total_cnt % 50 == 0:
            print(f"===> 지금까지 {total_cnt}개 수집 완료, 30초 동안 잠깐 쉰다...")
            time.sleep(30)

        time.sleep(random.uniform(1, 2.5))

    if total_cnt >= cnt:
        break

    # 다음 페이지 버튼 클릭
    try:
        next_page_num = str(x + 1)
        time.sleep(random.uniform(3, 5))
        driver.find_element(By.LINK_TEXT, next_page_num).click()
        print(f"===> {next_page_num} 페이지로 이동")
        time.sleep(random.uniform(4, 6))
    except Exception:
        print("다음 페이지 버튼을 찾지 못했거나, 더 이상 페이지가 없습니다.")
        break

f.close()

# DataFrame 구성 및 저장
g_best_seller = pd.DataFrame({
    '판매순위': ranking2,
    '이미지': [''] * len(ranking2),   # 이미지용 빈 컬럼 (엑셀에서 B열에 이미지 삽입)
    '상품이름': title2,
    '판매가격': s_price2,
    '별점': star_point2,
    '리뷰개수': review_cnt2,
    '상세내용': content2,
    '상세내용2': content_2,
    'url': url2
})

# csv 저장
g_best_seller.to_csv(fc_name, encoding="utf-8-sig", index=False)

# 엑셀 저장
g_best_seller.to_excel(fx_name, index=False)

print("\n")
print("=" * 80)
print("2.파일 저장 완료: txt 파일명 : %s " % ff_name)
print("3.파일 저장 완료: csv 파일명 : %s " % fc_name)
print("4.파일 저장 완료: xlsx 파일명 : %s " % fx_name)
print("=" * 80)

# 이미지 삽입하기 (엑셀 프로그램 없이)
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage

wb = load_workbook(fx_name)
ws = wb["Sheet1"] 

# 이미지 컬럼(B열) 설정
ws.column_dimensions['B'].width = 20

# 행 높이 조정
row_cnt = len(ranking2) + 1 

for r in range(2, row_cnt + 1):
    ws.row_dimensions[r].height = 100

# 이미지 삽입 (B열) – 실제 수집한 상품 수 기준
for i in range(1, len(ranking2) + 1):
    img_path = os.path.join(img_dir, f"{i}.jpg")
    cell_addr = f"B{i+1}"

    if not os.path.exists(img_path):
        continue

    img = XLImage(img_path)
    img.width = 120
    img.height = 100
    ws.add_image(img, cell_addr)

wb.save(fx_name)

# (선택) 크롬 드라이버 종료
driver.quit()

아마존 Best Seller 상품 정보 추출하기 


1.크롤링 할 건수는 몇건입니까?:  100
2.파일을 저장할 폴더명만 쓰세요(기본값:c:\py_temp2\): 


----------------------------------------------------------------------
1.url: https://www.amazon.pl/NIVEA-Active-Clean-M%C4%99ski-prysznic/dp/B098M367KG/ref=zg_bs_g_beauty_d_sccl_1/257-0814675-8474112?psc=1
2.판매순위: 1
3.제품이름: NIVEA MEN Active Clean Men's Shower Gel 3in1 - For face, body and hair Cleansing 500 ml
4.판매가격: 8.55 PLN
5.별점: 4.8
6.리뷰개수: 13,632
7.상세내용: Natural carbon – effectively removes impurities like a magnet     3-in-1 formula – for body, face and hair for convenience and time-saving     A refreshing, masculine scent – ​​provides a long-lasting feeling of cleanliness     Daily care without drying the skin - pH balanced for the skin     Environmentally friendly – ​​99% biodegradable formula, no microplastics
8.상세내용2: Mark       NIVEA           Element dimensions L x W x H       44 x 84 x 217 mm           Smell       Fresh           Age range (description)       Adult           Skin type       Normal           Product Benefits       Cleansing activated charcoal for all skin 